In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#共有 k 頁
df = pd.DataFrame(columns=["ID", "看板", "標題", "時間", "分數", "內容"])
count = 0
for k in range(1, 12):
    try:
        url = "https://www.ptt.cc/bbs/MobileComm/search?page=" + str(k) +"&q=索尼"
        print("正在處理", url)
        res = requests.get(url)
        htm = BeautifulSoup(res.text)
        t = htm.find_all("div", class_="r-ent")
        for i in t:
            t2 = i.find("div", class_="title")
            t3 = t2.find("a")
            url2 = "https://www.ptt.cc" + t3["href"]
            count += 1
            response = requests.get(url2)
            html = BeautifulSoup(response.text)
            content = html.find("div", id="main-content")
            imfo = content.find_all("span", class_="article-meta-value")
            m = content.find_all("div", class_="article-metaline")
            for i in m:
                i.extract()
            m = content.find_all("div", class_="article-metaline-right")
            for i in m:
                i.extract()
            print(content.text)
            push = content.find_all("div", class_="push")
            score = 0
            for p in push:
                tag = p.find("span", class_="push-tag").text
                if "推" in tag:
                    score += 1
                elif "噓" in tag:
                    score -= 1
                else:
                    score += 0
            s = pd.Series([imfo[0].text, imfo[1].text, imfo[2].text, imfo[3].text, score, content.text],
                          index=["ID", "看板", "標題", "時間", "分數", "內容"])
            df = df.append(s, ignore_index=True)
    except:
        continue
print("總共:", count, "個標題")
df.to_csv("ptt_sony_c.csv", encoding="utf-8", index=False)

In [0]:
df = pd.read_csv("./drive/My Drive/ptt_samsung.csv")
sorted(df["分數"], reverse=True)